# ASHITA K
# ADVANCE LEVEL TASK 2:
# 2) Next Word Prediction

# Importing Libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential, load_model 
from keras.layers import LSTM
from keras.layers.core import Dense, Activation 
from tensorflow.keras.optimizers import RMSprop
from nltk. tokenize import RegexpTokenizer 
import pickle
import heapq
import warnings
warnings.filterwarnings('ignore')

# Loading the data

In [2]:
data = '1661-0.txt'
text = open(data, encoding='utf-8').read().lower() 
print('corpus length:', len(text))

corpus length: 581888


In [3]:
tokenizer = RegexpTokenizer(r'\w+')
words = tokenizer.tokenize(text)

In [4]:
unique_words = np.unique(words)
unique_word_index = dict((c, i) for i, c in enumerate(unique_words))

In [5]:
WORD_LENGTH= 5
prev_words = [] 
next_words = []
for i in range(len (words) - WORD_LENGTH):
    prev_words.append(words[i:i + WORD_LENGTH])
    next_words.append(words[i + WORD_LENGTH])
print(prev_words[0]) 
print(next_words[0])

['project', 'gutenberg', 's', 'the', 'adventures']
of


In [6]:
X = np.zeros((len(prev_words), WORD_LENGTH, len(unique_words)), dtype=bool)
Y = np.zeros((len (next_words), len(unique_words)), dtype=bool)
for i, each_words in enumerate(prev_words):
    for j, each_word in enumerate(each_words):
        X[i, j, unique_word_index[each_word]] = 1
    Y[i, unique_word_index[next_words[i]]] = 1

In [7]:
print(X[0][0])

[False False False ... False False False]


In [8]:
model = Sequential()
model.add(LSTM(128, input_shape=(WORD_LENGTH, len(unique_words))))
model.add(Dense(len(unique_words)))   
model.add(Activation('softmax'))

In [9]:
optimizer = RMSprop(lr = 0.01) 
model.compile(loss = 'categorical_crossentropy', optimizer = optimizer, metrics=['accuracy'])
history = model.fit(X, Y, validation_split=0.05, batch_size=128, epochs=2, shuffle=True).history

Epoch 1/2
811/811 [==============================] - 222s 271ms/step - loss: 6.0169 - accuracy: 0.1054 - val_loss: 7.1298 - val_accuracy: 0.1007
Epoch 2/2
811/811 [==============================] - 215s 265ms/step - loss: 5.7900 - accuracy: 0.1472 - val_loss: 7.9124 - val_accuracy: 0.1034


In [10]:
model.save('keras_next_word_model.h5') 
pickle.dump(history, open("history.p", "wb")) 
model = load_model('keras_next_word_model.h5') 
history = pickle.load(open("history.p", "rb"))

In [11]:
def prepare_input(text):
    x = np.zeros((1, WORD_LENGTH, len(unique_words))) 
    for t, word in enumerate(text.split()):
        print (word)
        x[0, t, unique_word_index[word]] = 1
        return x 
prepare_input("Then how many are there?".lower())

then


array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]])

In [12]:
def sample(preds, top_n = 3):
    preds= np.asarray(preds).astype('float64') 
    preds= np.log(preds)
    exp_preds= np.exp(preds) 
    preds= exp_preds/np.sum(exp_preds) 
    return heapq.nlargest(top_n, range(len(preds)), preds.take)

In [13]:
def predict_completions(text, n=3):
    if text == "":
        return("")
    x = prepare_input(text) 
    preds = model.predict(x, verbose=0)[0]
    next_indices = sample(preds, n)
    return [unique_words[idx] for idx in next_indices]

# Next word prediction

In [15]:
s = "you have frequently seen the steps which lead up from the hall to this room" 
print("correct sentence: ",s) 
seq = " ".join(tokenizer.tokenize(q.lower())[0:5]) 
print("Sequence: ",seq) 
print("next possible words: ", predict_completions(seq, 5))

correct sentence:  you have frequently seen the steps which lead up from the hall to this room
Sequence:  you have frequently seen the
you
next possible words:  ['in', 'and', 'to', 'of', 'i']
